In [ ]:
from datetime import datetime
import MetaTrader5 as mt5
import numpy as np

mt5.initialize()

rates = mt5.copy_rates_from('EURUSD', mt5.TIMEFRAME_M10, datetime(2019, 11, 2, 19), 20)

mt5.shutdown()


In [ ]:
# Only trade low period
# Only trade on high volume time
# Only follow trend
# Get out quick
# High frequency trading
# Hedge. Multiple trades

# Trend: Up / Down / Side / Mixed
# Strength: Strong / Light
# Switch: Up->Down / Down->Up / Up->Side / Down->Side / Side->Up / Side->Down
# Length: Long / Short


In [ ]:
import pandas as pd

df = pd.DataFrame(rates).loc[:, [
    'time', 'open', 'high', 'low', 'close', 'tick_volume']]
df = df.rename(columns={'time': 'Time', 'high': 'High', 'low': 'Low',
                        'close': 'Close', 'tick_volume': 'Volume', 'open': 'Open'})

df['Time'] = pd.to_datetime(df['Time'], unit='s')
df['Weekday'] = np.array([x.weekday() for x in df['Time']])
df['Weekday'] = np.array([x.weekday() for x in df['Time']])
df['Weekday']


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.figure as mpfg
import mplfinance as mpf

import pandas as pd


def _return(d: pd.Series):
    return d.iloc[-1]/d.iloc[0]-1


def _slope(d: pd.Series):
    sl = (d.iloc[-1] - d.iloc[:d.size-1]) / np.arange(d.size+100, 101, -1)
    return sl.iloc[np.argmax(abs(sl))]


def _returnM(d: pd.Series):
    sl = (d.iloc[-1] / d.iloc[:d.size-1]) - 1
    return sl.iloc[np.argmax(abs(sl))]
    return sl.mean()


def _slopeV(d: pd.Series):
    sl = (d.iloc[-1] - d.iloc[:d.size-1]) / np.arange(d.size, 1, -1)
    return sl.std()




df = df.set_index('Time')


df['Return'] = df['Open'].rolling(24).apply(_return)
df['Slope'] = df['Open'].rolling(48).apply(_slope)
df['SlopeV'] = df['Open'].rolling(48).apply(_slopeV)
df['VarH'] = (df['High'] - df['Low'] +
              abs(df['Open'] - df['Close']) * 2).rolling(48).std()
df['VarP'] = df['Open'].rolling(48).std()

# df['Return4'] = df['Open'].rolling(4).apply(_return)
# df['Return24'] = df['Open'].rolling(24).apply(_return)
# df['Return48'] = df['Open'].rolling(48).apply(_return)


df['ReturnM'] = df['Open'].rolling(48).apply(_returnM)



df['SellX'] = df['Open']
df['SellX'][(df['Slope'] < 0.5) | (df['ReturnM'] < 0.8)] = np.nan
df['BuyX'] = df['Open']
df['BuyX'][(df['Slope'] > -0.5) | (df['ReturnM'] > -0.8)] = np.nan


df['Sell'] = df['Open']
df['Sell'][df['Slope'] < 0.5] = np.nan
df['Buy'] = df['Open']
df['Buy'][df['Slope'] > -0.5] = np.nan

df = df[48:]

apds = [
    # mpf.make_addplot(tcdf),
    # mpf.make_addplot(low_signal, type='scatter', markersize=200, marker='^'),
    # mpf.make_addplot(high_signal, type='scatter', markersize=200, marker='v'),
    # mpf.make_addplot(df['Return'], panel=1, color='royalblue'),
    mpf.make_addplot(df['Slope'], panel=1, color='red', secondary_y=False),
    mpf.make_addplot(df['Return'], panel=1, color='royalblue', secondary_y=True),
    mpf.make_addplot(df['ReturnM'], panel=1, color='green', secondary_y=True),
    # mpf.make_addplot(df['Slope']X, panel=1, color='royalblue'),
    # mpf.make_addplot(df['VarH'], panel=1, color='yellow', secondary_y=True),
    # mpf.make_addplot(df['VarP'], panel=1, color='lime', secondary_y=True),
]

if np.all(df['Sell'] == np.nan):
    apds.append(mpf.make_addplot(df['Sell'], type='scatter',markersize=100, marker='v', color='red'))

if np.all(df['Buy'] == np.nan):
    apds.append(mpf.make_addplot(df['Buy'], type='scatter', markersize=100, marker='^', color='blue'))

if np.all(df['SellX'] == np.nan):
    apds.append(mpf.make_addplot(df['SellX'], type='scatter', markersize=50, marker='v', color='orange'))

if np.all(df['BuyX'] == np.nan):
    apds.append(mpf.make_addplot(df['BuyX'], type='scatter', markersize=50, marker='^', color='cyan'))

fig, axes = mpf.plot(df, addplot=apds, figratio=(24, 10), type='candle', style='nightclouds', volume=False,
                     datetime_format='%Y-%m-%d %H:%M', xrotation=90, tight_layout=True, returnfig=True)
axes[0].locator_params(nbins=50, axis='x')
plt.show()
